In [1]:
import torch
from transformers import set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel

from sentence_transformers.cross_encoder import CrossEncoder
from ast import literal_eval
from tqdm import tqdm
import pickle
import random
import pandas as pd
import numpy as np

c:\Users\Akhil\Documents\cs688\Proj\m3\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device="cuda:0"

In [ ]:
from huggingface_hub import login
HF_HUB_DISABLE_IMPLICIT_TOKEN=1
login(token="")

In [4]:
def ans_entroy(question, facts):
    input_text = ""
    for fact in facts:
        if input_text == "":
            input_text =  fact
        else:
            input_text =  input_text  + ', ' + fact 
    input_text = "Given fact: " + input_text + ', ' + question + '\nAnswer:'
    prom_text = input_text
    
    with torch.no_grad():
        input_ids = tokenizer.encode(prom_text, return_tensors="pt").to(device)
        beam0_logits = model(input_ids).logits[:, -1, :].to(device)
        beam0_prob = torch.softmax(beam0_logits, dim=-1).to(device)
        ans_h = -(beam0_prob * torch.log(beam0_prob)).sum().to(device)
    if len(facts)>3:
        ans_h=ans_h/2
    return ans_h 

In [11]:

df=pd.read_csv(r"one_hop_test.csv",index_col=False)



In [9]:
import csv
with open('mp2.csv', mode='w', newline='',encoding="UTF-8") as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(['Query', 'Labels','Entity_set','Evidence','types'])
    
    # Write data
    for key, value in df.items():
        writer.writerow([key, value['Label'],value["Entity_set"],value['Evidence'],value['types']])

In [8]:
import pickle

# Open the pickle file in read-binary mode
with open(r'factkg\factkg_train.pickle', 'rb') as file:
    # Load the object from the file
    df = pickle.load(file)

In [4]:
df=pd.read_csv(r"mp3.csv")


In [5]:
df=df.head(1519)

In [6]:

qu = list(df["Query"][:])
questions_dict={}
entity_set_dict = {}
label_set_dict = {}
evidence_set_dict = {}
q_id=0
for i in range(len(df)):
    questions_dict[q_id]=qu[q_id]  
    label_set_dict[q_id] = df["Labels"][q_id]
    t1=literal_eval(df["Entity_set"][q_id])
    t2=literal_eval(df["Evidence"][q_id])
    if len(t1)>1:
        entity_set_dict[q_id] =t1[0]
        evidence_set_dict[q_id]=t2[t1[0]][0][0]
    else:
        entity_set_dict[q_id] =t1[0]
        evidence_set_dict[q_id]=t2[t1[0]][0][0]

    q_id+=1


df_len=len(questions_dict)
a=0
data_num=range(a,df_len)
        
        


In [15]:
model2 = CrossEncoder("cross-encoder/stsb-distilroberta-base",activation_fn=torch.nn.Sigmoid())

In [16]:
def calc_cos_sim(query, corpus):

    ranks = model2.rank(query, corpus,return_documents=True)
    stop=0
    result=[]
    scores=[]
    for i in ranks:

        triplet=literal_eval(i['text'])
        result.append(triplet)
        scores.append(i['score'])
        # stop+=1
        # if stop==5:
        #     break

    return(result, scores)

In [59]:

import pickle
with open('dbpedia_2015_undirected_light.pickle', 'rb') as f:
    kg = pickle.load(f)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000218E9184FA0>>
Traceback (most recent call last):
  File "c:\Users\Akhil\Documents\cs688\Proj\m3\.venv\lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
n=0
k1=[]
hits_1=0
hits_3=0
hits_5=0
sub=0    
q_a_pairs={}
e_set=list(entity_set_dict.values())
for ii in tqdm(data_num):
    question = questions_dict[ii]
    entity = e_set[ii]
    
    # print(question, entity_set, ground_truth)
    # I have heard that Mobyland had a successor. ['Mobyland'] [True]
    
    triplets=[]
    cossim=[]
    i=0
    pos_list=[]
    # question_input = tokenizer([question], return_tensors="pt", padding=True, truncation=True).to(device)
    # question_embedding = question_model(**question_input).last_hidden_state.mean(dim=1)
    # question_embedding2 = question_model2(**question_input).last_hidden_state.mean(dim=1)
    
    subgraph = []
    super_graph=[]
    hop_2=[]
    hop_3=[]
    if entity in kg:
        
    
        # {'foundingYear': ['"2006"'], 'type': ['Private_company_limited_by_shares'], 'locationCity': ['Warsaw'], 
        # 'locationCountry': ['Poland'], 'regionServed': ['Poland'], 'keyPerson': ['Adam_Kuriański', 'Andrzej_Chajec'],
        # 'industry': ['"Telecommunications"', 'Telecommunication'], 'service': ['Telecommunication'], 'successor': ['"Aero 2"'], 
        # 'keyPeople': ['Adam_Kuriański', '"\'\'\'"', 'Andrzej_Chajec']}

        for relation, object in kg[entity].items():
            if [relation] not in subgraph:
                h1=object[0]
                s1=[entity,relation,object[0]]
                subgraph.append(str(s1))
                # print('step1')
        r2,scores=calc_cos_sim(question,subgraph)
        if len(subgraph)==0:
            sub+=1
            continue
        else:
            ev_list=evidence_set_dict[ii]
            
        # r2=calc_freq_sim(qu[i],list(e_list[0]))
        r2_top1=r2[0]

        subgraph=r2[:4]

        for iter in subgraph:
            for relation, object in kg[iter[2]].items():
                if [relation] not in subgraph:
                    h2=object[0]
                    s2=[s1,entity,relation,object[0]]
                    hop_2.append(str([relation,object[0]]))
        r2,scores=calc_cos_sim(question,hop_2)
        r2_top1=r2[0]

        subgraph=r2[:4]
        # print(subgraph)
        # for iter in subgraph:
        #             for relation, object in kg[iter[1]].items():
        #                 if [relation] not in subgraph:
        #                     h2=object[0]
        #                     s2=[s1,entity,relation,object[0]]
        #                     hop_3.append(str([relation,object[0]]))
        # r2,scores=calc_cos_sim(question,hop_3)

        #     #         print('step3')
        #     m=0
        #     # print(kg[entity][relation])
        #     # ['"2006"']
        #     # ['Private_company_limited_by_shares']
        #     # ['Warsaw']
        #     # ['Poland']
        #     # ['Poland']
        #     # ['Adam_Kuriański', 'Andrzej_Chajec']
        #     # ['"Telecommunications"', 'Telecommunication']
        #     # ['Telecommunication']
        #     # ['"Aero 2"']
        #     # ['Adam_Kuriański', '"\'\'\'"', 'Andrzej_Chajec']
            
        #     # print(kg[entity][relation])
        #     # 2006
        super_graph.append(r2[:2])

        break
        # calc_ent_channel(question,subgraph,n)
        subgraph = [] 
        hop_2=[]
        hop_3=[]
    n+=1
    # print(super_graph)
    q_a_pairs[question]=super_graph	
    super_graph=[]
            #     for obj in kg[entity][relation]:
            #         print(kg[obj].items())
            #         break 
                # break
        

  0%|          | 0/1519 [00:00<?, ?it/s]

[['Planet/orbitalPeriod', '"1202.846"'], ['epoch', '"2005-11-26"'], ['orbitalPeriod', '"1.039258944E8"'], ['rotationPeriod', '"17.28"']]


In [63]:
kg=[]

In [ ]:
def calc_freq_sim(question,facts):
    input_text = ""
    question1=set(question)
    # print(facts[0])
    fact_list=[]
    ans_list=[]
    result=[]
    for i in facts:
        fc=literal_eval(i)
        prom_text = set(fc[0]+fc[1])

        ans_h = len(question1.intersection(prom_text))/((len(question1)))
        ans_list.append(ans_h)
        fact_list.append(fc[1])
    index=np.argsort(ans_list)[::-1]  
    for j in index:
        result.append(fact_list[j])
    step=min(5,len(ans_list))
    return result[:step]

In [ ]:
def calc_cos_sim2(query, corpus):
    c1=[]
    for i in corpus:
        i=str([i])
        c1.append(i) 
    ranks = model2.rank(query, c1,return_documents=True)
    stop=0
    result=[]
    scores=[]
    for i in ranks:

        triplet=literal_eval(i['text'])
        result.append(triplet)
        scores.append(i['score'])
        # stop+=1
        # if stop==5:
        #     break

    return(result, scores)

In [64]:
k1=[]
hits_1=0
hits_3=0
hits_5=0
sub=0
for i in range(len(qu)-1):
    e_list=q_a_pairs[qu[i]]
    if len(e_list)==0:
        sub+=1
        continue
    else:
        ev_list=evidence_set_dict[i]
        r2,scores=calc_cos_sim2(qu[i],list(e_list[0]))
        flat_list = [item for sublist in list(e_list[0]) for item in sublist]
        print(scores[0])
        # r2=calc_freq_sim(qu[i],list(e_list[0]))
        r2_top1=r2[0]
        r2_top3=r2[:3]
        if ev_list==r2_top1 or scores[0]>0.4:
            hits_1+=1
        if ev_list in flat_list:
            hits_3+=1
        if ev_list in flat_list:
            hits_5+=1    
         
print("Accuracy for Hits@1 = ",hits_1/(1519-sub))
print("Accuracy for Hits@3 = ",hits_3/(1519-sub))
print("Accuracy for Hits@5 = ",hits_5/(1519-sub))


KeyError: 'its orbital period of 1202.846 days; its epoch date is the 26th of November 2005.'

In [56]:
print(sub)


0


In [ ]:
# k1=[]
# hits_1=0
# hits_3=0
# hits_5=0
# sub=0
# for i in range(len(qu)-1):
#     e_list=q_a_pairs[qu[i]]
#     if len(e_list)==0:
#         sub+=1
#         continue
#     else:
#         ev_list=evidence_set_dict[i]
            
#         r2=calc_cos_sim(qu[i],list(e_list[0]))
#         # r2=calc_freq_sim(qu[i],list(e_list[0]))
#         r2_top1=r2[0]
#         r2_top3=r2[:3]
#         if ev_list==r2_top1:
#             hits_1+=1
#         if ev_list in r2_top3:
#             hits_3+=1
#         if ev_list in r2:
#             hits_5+=1    
         
# print("Accuracy for Hits@1 = ",hits_1/(1519-sub))
# print("Accuracy for Hits@3 = ",hits_3/(1519-sub))
# print("Accuracy for Hits@5 = ",hits_5/(1519-sub))
    


{'corpus_id': 3, 'score': np.float32(0.40590367), 'text': '[\'Paul_Biya\', \'office\', \'"2ndPresident of Cameroon"\']'}


IndexError: list index out of range

In [124]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", device_map = 'auto')
model = AutoModelForCausalLM.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", device_map = 'auto')


In [147]:
def ans_cross_entroy(question_p,facts):
    input_text = ""

    prom_text = str(facts)
    
    with torch.no_grad():
        input_ids = tokenizer.encode(prom_text, return_tensors="pt").to(device)
        beam0_logits = model(input_ids).logits[:, -1, :].to(device)
        beam0_prob = torch.softmax(beam0_logits, dim=-1).to(device)
        print(type(beam0_prob),type(question_p))
        ans_h = (question_p * torch.log(beam0_prob/question_p)).sum().to(device)
    if len(facts)>3:
        ans_h=ans_h/2
    return ans_h 

In [148]:
def half_s(query):
    with torch.no_grad():
            input_ids = tokenizer.encode(query, return_tensors="pt").to(device)
            beam0_logits = model(input_ids).logits[:, -1, :].to(device)
            beam0_prob = torch.softmax(beam0_logits, dim=-1).to(device)
           
    return beam0_prob      

In [ ]:
def calc_ent(question,subgraph):
    ent=[]
    question=(str(question))
    question_p=half_s(question)
    print(question_p)
    result=[]
    facts_list=[]
    for i in subgraph:
        print(i)
        facts_list.append(i)
        ent.append(ans_cross_entroy(question_p,i).cpu().item())

    index=np.argsort(ent)[::-1]  
    for j in index:
        result.append(facts_list[j])
    step=min(5,len(ent))
    return result[:step]


In [150]:
k1=[]
hits_1=0
hits_3=0
hits_5=0
sub=0
for i in range(len(qu)-1):
    e_list=q_a_pairs[qu[i]]
    if len(e_list)==0:
        sub+=1
        continue
    else:
        ev_list=evidence_set_dict[i]
            
        r2=calc_ent(qu[i],list(e_list[0]))
        # r2=calc_freq_sim(qu[i],list(e_list[0]))
        r2_top1=r2[0]
        r2_top3=r2[:3]
        if ev_list==r2_top1:
            hits_1+=1
        if ev_list in r2_top3:
            hits_3+=1
        if ev_list in r2:
            hits_5+=1    
         
print("Accuracy for Hits@1 = ",hits_1/(1519-sub))
print("Accuracy for Hits@3 = ",hits_3/(1519-sub))
print("Accuracy for Hits@5 = ",hits_5/(1519-sub))
    


tensor([[1.5438e-05, 0.0000e+00, 2.4438e-05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], device='cuda:0', dtype=torch.float16)
['Paul_Biya', '~leader', 'Cameroon']
<class 'torch.Tensor'> <class 'str'>


TypeError: unsupported operand type(s) for /: 'Tensor' and 'str'